<a href="https://colab.research.google.com/github/dudaholandah/sibgrapi2022_wuw/blob/main/sibigrapi_hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!apt-get install poppler-utils 
!pip install pdf2image
!pip install unidecode
!pip install umap-learn[plot]
!pip install trimap
!pip install -U kaleido
!pip install scikit-learn-extra

In [2]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn import preprocessing
from pandas.core.frame import DataFrame
import plotly.express as px
from sklearn.manifold import TSNE
import re
import umap
import trimap
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import Isomap
from sklearn.manifold import trustworthiness
import plotly.graph_objects as go
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
import plotly
import pdf2image
import os
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn_extra.cluster import KMedoids

# Pre-Processing Data

In [3]:
data = pd.read_excel("vegan_dataset.xlsx", sheet_name='Veganos')

In [4]:
data.head()

,Classification,Description,Ingredients,Serving Size,Kcal,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,Dietary Fiber,Sodium,B12,Calcium,Zinc
0,MEAT 2,Carne Moída do Futuro,"water, texturized soy protein, pea protein, ch...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,0.0,0.0
1,MEAT 3,Carne Moída do Futuro,"water, texturized soy protein, isolated soy pr...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,0.0,0.0
2,POULTRY 3,Frango do Futuro,"water, texturized soy protein, isolated soy pr...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,0.0,0.0
3,PORK 1,Linguiça do Futuro,"water, texturized soy protein, isolated soy pr...",50.0,139.0,7.7,0.0,5.7,9.5,3.9,0.2,294.0,0.0,0.0,0.0
4,MEAT 1,Hamburguer de Soja Goshen,"texturized soy protein, water, soy oil, modif...",50.0,118.0,2.8,0.0,7.0,9.0,1.3,2.0,344.0,0.0,0.0,0.0


### Separating data

In [5]:
data_ingredients = pd.DataFrame(data['Ingredients'])
data_nutrients = data.drop(columns=['Ingredients', 'Classification', "Description"])
data_clasification = pd.DataFrame(data['Classification'])
data_name = pd.DataFrame(data["Description"])
label_data = data_clasification.join(data_name)

In [6]:
data_nutrients.head()

,Serving Size,Kcal,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,Dietary Fiber,Sodium,B12,Calcium,Zinc
0,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,0.0,0.0
1,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,0.0,0.0
2,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,0.0,0.0
3,50.0,139.0,7.7,0.0,5.7,9.5,3.9,0.2,294.0,0.0,0.0,0.0
4,50.0,118.0,2.8,0.0,7.0,9.0,1.3,2.0,344.0,0.0,0.0,0.0


### Normalizing Nutrient Values (MinMax Scaler) 

In [7]:
X = data_nutrients.values
print(X)

[[ 80.  168.   11.  ...   0.    0.    0. ]
 [ 80.  168.   11.  ...   0.    0.    0. ]
 [ 80.  168.   11.  ...   0.    0.    0. ]
 ...
 [ 40.   84.    5.2 ...   0.    0.    0. ]
 [ 10.   25.    3.3 ...   0.    0.    0. ]
 [  7.   28.    1.2 ...   0.    0.    0. ]]


In [8]:
attributes_dummies = data_nutrients.columns
normalize = preprocessing.MinMaxScaler()
xscaled = normalize.fit_transform(X)

nutrients_normalized = pd.DataFrame(xscaled,columns=attributes_dummies)
nutrients_normalized = nutrients_normalized.replace(np.nan,0)

nutrients_normalized.head()

,Serving Size,Kcal,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,Dietary Fiber,Sodium,B12,Calcium,Zinc
0,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,0.0,0.0
1,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,0.0,0.0
2,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,0.0,0.0
3,0.125364,0.212121,0.171111,0.0,0.135714,0.193878,0.211957,0.002128,0.310782,0.0,0.0,0.0
4,0.125364,0.174688,0.062222,0.0,0.166667,0.183673,0.070652,0.021277,0.363636,0.0,0.0,0.0


### Normalizing Ingredients (One Hot Encode)

In [9]:
def pre_process(text):
    text = re.sub(r'[.,():%-]+', " ", text)
    text = re.sub(r'[\s]+', " ", text)
    text = unidecode(text.strip().lower())
    return text

In [ ]:
vocab = set()

for i in range(data_ingredients.shape[0]):
  aux = data_ingredients.at[i,'Ingredients'].split(",")
  for word in aux:
    word = pre_process(word)
    if len(word) > 1: vocab.add(word.strip())

In [12]:
ingredients_normalized = pd.DataFrame()

for i in range(276):
  aux = data_ingredients.at[i,'Ingredients'].split(",")
  for word in aux:
    word = pre_process(word)
    ingredients_normalized.at[i,word] = 1

ingredients_normalized = ingredients_normalized.replace(np.nan,0)
ingredients_normalized.head()

,water,texturized soy protein,pea protein,chickpea flour,vegetal fat,modified starch,onion,meat flavored condiment,salt,sugar,...,tricalcium phosphatxanthan gum,phosphoric acid,aquafaba,potassium sorbate disodium calcium,avocado,potassium chloride,isolated vegetable protein,soy,pea starch,isolated rice protein
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preparing data

In [13]:
raw_data = pd.DataFrame(ingredients_normalized)
raw_data = pd.concat([raw_data,nutrients_normalized], axis=1)
raw_data = raw_data.replace(np.nan,0)

raw_data.head()

,water,texturized soy protein,pea protein,chickpea flour,vegetal fat,modified starch,onion,meat flavored condiment,salt,sugar,...,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,Dietary Fiber,Sodium,B12,Calcium,Zinc
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.171111,0.0,0.135714,0.193878,0.211957,0.002128,0.310782,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.062222,0.0,0.166667,0.183673,0.070652,0.021277,0.363636,0.0,0.0,0.0


In [14]:
X = raw_data.values
X_ingredients = ingredients_normalized.values
X_nutrients = nutrients_normalized.values
y = label_data.values
all_data = label_data.join(raw_data)

for i in range(276):
  label = label_data['Classification'][i].rstrip()
  label_data.at[i,'Classification'] = label
  all_data.at[i,'Classification'] = label

print(all_data['Classification'].value_counts())

DAIRY 1      80
DAIRY 3      45
MEAT 1       34
MEAT 3       18
POULTRY 1    14
EGG 1        14
POULTRY 3    13
PORK 1       12
DAIRY 2      11
MEAT 2        9
FISH 1        8
PORK 2        6
POULTRY 2     5
FISH 2        5
EGG 2         2
Name: Classification, dtype: int64


# Finding Best Hyper Parameters

In [41]:
def coeficiente_silhueta(X_high,X_low):
  silhouette_avg_high = []
  silhouette_avg_low = []

  for k in range(3,20):
    clusterer = KMedoids(n_clusters=k, init='k-medoids++',random_state=10)
    cluster_labels_high = clusterer.fit_predict(X_high)
    cluster_labels_low = clusterer.fit_predict(X_low)
    silhouette_avg_high.append(silhouette_score(X_high, cluster_labels_high))
    silhouette_avg_low.append(silhouette_score(X_low, cluster_labels_low))

  mean_silhouete_high = np.mean(np.array(silhouette_avg_high))
  mean_silhouete_low = np.mean(np.array(silhouette_avg_low))
  std_silhouete_high = np.std(np.array(silhouette_avg_high))
  std_silhouete_low = np.std(np.array(silhouette_avg_low))
    
  return mean_silhouete_high, mean_silhouete_low, std_silhouete_high, std_silhouete_low

### t-SNE

In [ ]:
kneigh = []
y_tsne_precision = np.zeros((10,21))
y_tsne_silhouete = np.zeros((10,21))

v_perplexity = [5,10,15,20,25,30,35,40,45,50]
v_learning_rate = [10,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000]

for pv in range(0,21):
  kneigh.append(v_learning_rate[pv])
  for pi in range(0,10):
    tsne = TSNE(n_components=2,perplexity=v_perplexity[pi],learning_rate=v_learning_rate[pv],metric='euclidean', init='pca')
    X_tsne = tsne.fit_transform(X)
    X_tsne[1:4, :]
    precision = trustworthiness(X, X_tsne, metric='euclidean')
    mean_high,mean_low,std_high,std_low = coeficiente_silhueta(X,X_tsne)
    y_tsne_precision[pi][pv] = precision
    y_tsne_silhouete[pi][pv] = mean_low

In [43]:
fig = go.Figure()
fig.update_layout(title_text='Trustworthiness', title_x=0.5)

for pi in range(10):
  fig.add_trace(go.Scatter(x=kneigh, y=y_tsne_precision[pi],
                    mode='lines+markers',
                    name='t-SNE perplexity ' + str(v_perplexity[pi])))

fig.show()

In [44]:
fig = go.Figure()
fig.update_layout(title_text='Silhouette Coefficient', title_x=0.5)
for pi in range(10):
  fig.add_trace(go.Scatter(x=kneigh, y=y_tsne_silhouete[pi],
                    mode='lines+markers',
                    name='t-SNE perplexity ' + str(v_perplexity[pi])))

fig.show()

### UMAP

In [46]:
kneigh = []
v_neighbors = [10, 20, 50, 100, 120, 150, 200]
v_mindist = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
y_umap_precision = np.zeros((7,11))
y_umap_silhouete = np.zeros((7,11))

for pv in range(0,11):
  kneigh.append(v_mindist[pv])
  for pi in range(0,7):
    reducer = umap.UMAP(n_components=2, n_neighbors=v_neighbors[pi], min_dist=v_mindist[pv], metric='euclidean')
    X_umap = reducer.fit_transform(X)
    X_umap[1:4, :]
    precision = trustworthiness(X, X_umap, metric='euclidean')
    mean_high,mean_low,std_high,std_low = coeficiente_silhueta(X,X_umap)
    y_umap_precision[pi][pv] = precision
    y_umap_silhouete[pi][pv] = mean_low

In [47]:
fig = go.Figure()
fig.update_layout(title_text='Trustworthiness', title_x=0.5)
for pi in range(0,7):
  fig.add_trace(go.Scatter(x=kneigh, y=y_umap_precision[pi],
                    mode='lines+markers',
                    name='UMAP n_neighbors ' + str(v_neighbors[pi])))

fig.show()

In [48]:
fig = go.Figure()
fig.update_layout(title_text='Silhouette Coefficient', title_x=0.5)
for pi in range(0,7):
  fig.add_trace(go.Scatter(x=kneigh, y=y_umap_silhouete[pi],
                    mode='lines+markers',
                    name='UMAP n_neighbors ' + str(v_neighbors[pi])))

fig.show()

### PCA

In [49]:
components = [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
kneigh = []
y_pca_precision = []
y_pca_silhouete = []

for n in components:
  kneigh.append(n)
  pca = sklearnPCA(n_components=n)
  X_pca = pca.fit_transform(X)
  X_pca[1:4, :]
  precision = trustworthiness(X, X_pca, n_neighbors=1, metric='euclidean')
  mean_high,mean_low,std_high,std_low = coeficiente_silhueta(X,X_pca)
  y_pca_precision.append(precision)
  y_pca_silhouete.append(mean_low)

In [50]:
fig = go.Figure()

fig.update_layout(title_text='Trustworthiness', title_x=0.5)
fig.add_trace(go.Scatter(x=kneigh, y=y_pca_precision,
                  mode='lines+markers',
                  name='PCA'))

fig.show()

In [51]:
fig = go.Figure()

fig.update_layout(title_text='Silhouette Coefficient', title_x=0.5)
fig.add_trace(go.Scatter(x=kneigh, y=y_pca_silhouete,
                  mode='lines+markers',
                  name='PCA'))

fig.show()

### TRIMAP

In [52]:
v_random = [1,2,3,4,5,6,7,8,9,10]
v_inliers = [5,10,15,20,25,30,35,40]
kneigh = []
y_trimap_precision = np.zeros((8,10))
y_trimap_silhouete = np.zeros((8,10))

for pv in range(0,10):
  kneigh.append(v_random[pv])
  for pi in range(0,8):
    X_trimap = trimap.TRIMAP(n_dims=2, n_inliers=v_inliers[pi], n_random=v_random[pv]).fit_transform(X)
    X_trimap[1:4, :]
    precision = trustworthiness(X, X_trimap, metric='euclidean')
    mean_high,mean_low,std_high,std_low = coeficiente_silhueta(X,X_trimap)
    y_trimap_precision[pi][pv] = precision
    y_trimap_silhouete[pi][pv] = mean_low

In [53]:
fig = go.Figure()
fig.update_layout(title_text='Trustworthiness', title_x=0.5)

for pi in range(0,8):
  fig.add_trace(go.Scatter(x=kneigh, y=y_trimap_precision[pi],
                    mode='lines+markers',
                    name='TRIMAP inliners ' + str(v_inliers[pi])))

fig.show()

In [55]:
fig = go.Figure()
fig.update_layout(title_text='Silhouette Coefficient', title_x=0.5)

for pi in range(0,8):
  fig.add_trace(go.Scatter(x=kneigh, y=y_trimap_silhouete[pi],
                    mode='lines+markers',
                    name='TRIMAP inliners ' + str(v_inliers[pi])))

fig.show()